In [ ]:
using Plots, LinearAlgebra

In [ ]:
cd(s"C:\Users\User\Desktop\Mycop\plots")

https://doi.org/10.1119/10.0000189

$$
\vec \lambda = \frac{\vec n + (\gamma-1)(\vec n\cdot\vec\beta)\vec \beta + \gamma\vec\beta}{\gamma(1 + \vec\beta\cdot\vec n)}
$$

$$
\vec E = \frac{\Delta\vec r/\gamma^2 + \Delta\vec r(\vec R\cdot\vec a) - \vec a(\vec R\cdot\Delta\vec r)}{\left((t-t^\prime)-\vec R\cdot\vec\beta\right)^3}
$$

$\Delta\vec r = \vec R - \vec r_0 - (t-t^\prime)\vec\beta$  
$t - t^\prime = r = \sqrt{x^2+y^2}$  
$t^\prime = t - \sqrt{x^2+y^2}$

In [ ]:
function E(r, t)
    r -= x₀(t)
    t′ = t - hypot(r...)
    β = v₀ₓ(t′)
    a = a₀ₓ(t′)
    γ = 1/sqrt(1 - β ⋅ β)
    Δr = r - (t-t′)*β
    vec_E =  ( Δr/γ^2 + Δr*(r ⋅ a) - a*(r ⋅ Δr) ) / ( t-t′ - r ⋅ β )^3
    hypot(vec_E...)
end

function λ(t, t′, ϕ)
    n = [cos(ϕ) sin(ϕ)]
    β = v₀ₓ(t′) 
    nβ = n ⋅ β
    γ = 1/sqrt(1-β ⋅ β)
    vec_λ = ( n + (γ-1)*nβ*β + γ*β ) / ( γ*(1+nβ) )
    return x₀(t′) + (t-t′)*vec_λ
end;

### Bremsstrahlung

In [ ]:
# двукомпонентные вектора
x₀(t) = begin
    [t<=0 ? t/√2 :
    0<t<=1 ? 1 + t/√2 - sqrt(1+t^2) :
    1 - 1/√2 0.0]
end

v₀ₓ(t) = begin
    [t<=0 ? 1.0/√2 :
    0<t<=1 ? 1.0/√2 - t/sqrt(1+t^2) : 0.0 0.0]
end

a₀ₓ(t) = begin
    [0<t<=1 ? -1.0/sqrt(1+t^2)^3 : 0.0 0.0]
end;

### Oscillator

In [ ]:
x₀(t)  = [ 0.0 t<0 ? 0.0 : 0.8/π * sin(π*t) ]
v₀ₓ(t) = [ 0.0 t<0 ? 0.0 : 0.8cos(π*t) ]
a₀ₓ(t) = [ 0.0 t<0 ? 0.0 : -0.8π * sin(π*t) ];

In [ ]:
tt = -1:0.01:2
ploter(fun) = plot(tt, vcat(fun.(tt)...), lab = "$(fun)")

plot(ploter(x₀), ploter(v₀ₓ), ploter(a₀ₓ), line = 3 )

In [ ]:
t = 3.0
X = range(-4, stop = 4, length = 128)
Y = range(-4, stop = 4, length = 128)  
Z = [ E([x y], t ) for y in Y, x in X ];

In [ ]:
maximum(Z)

In [ ]:
heatmap(X, Y, log10.(Z))

In [ ]:
heatmap(X, Y, clamp.(log10.(Z), -2, 2))

In [ ]:
heatmap(X, Y, clamp.(log10.(Z), -2, 2), # oбрезание
    legend = false, ticks = nothing, border = :none, size = (512, 256), c = :devon  ) # :grays

for fi in π/6:π/6:2π
    zz = vcat( [λ(t, tt, fi) for tt in t-10:0.1:t ]... );
    plot!(zz[:,1], zz[:,2], line = (1, :yellow), )
end
scatter!([x₀(t)[1]],[x₀(t)[2]], c = :red)
vline!([0.0], c = :green, line = (1, :dash))
yaxis!((Y[1], Y[end]))
xaxis!((X[1], X[end]))

In [ ]:
@time for (i, t) in enumerate(range(0, stop = 8, length = 64)) # картинки для гифки
    Z = [ E([x y], t) for y in Y, x in X ];
    heatmap(X, Y, clamp.(log10.(Z), -2, 2), 
        legend = false, ticks = nothing, border = :none, size = (512, 256)  ) # , c = :grays

    for fi in π/6:π/6:2π # t-10 на сколько назад во времени
        zz = vcat( [λ(t, tt, fi) for tt in t-10:0.01:t ]... )  # 0.01 четкость линий поля
        plot!(zz[:,1], zz[:,2], line = (1, :yellow), )
    end
    scatter!([x₀(t)[1]],[x₀(t)[2]], c = :red #= , title = "t = $(round(t, digits = 1))" =# )
    #vline!([0.0], c = :green, line = (1, :dash))
    yaxis!((Y[1], Y[end]))
    xaxis!((X[1], X[end]))

    savefig("img$i.png")
end

http://docs.juliaplots.org/latest/animations/

## Синхротрон

https://cyberleninka.ru/article/n/problemy-teorii-atoma-vodoroda-i-fotonnoe-vzaimodeystvie-elektrona-i-protona

https://cyberleninka.ru/article/n/sinhrotron-kak-makroatom

In [ ]:
function λ₂(t, t′, ϕ)
    β = radi*omeg
    γ = 1/sqrt(1-β^2)
    ψt = omeg*t′ + 2atan( β-tan( 0.5γ*omeg*t′ + atan( γ*( β-tan(0.5ϕ) ) ) )/γ )
    
    return x₀(t′) + (t-t′)*[cos(ψt) sin(ψt)]
end;

radi = 0.2
omeg = 0.4/radi
peri = 2π/omeg

x₀(t) = radi*[cos(omeg*t) sin(omeg*t)]
v₀ₓ(t) = radi*omeg*[-sin(omeg*t) cos(omeg*t)]
a₀ₓ(t) = -omeg^2 * x₀(t);

In [ ]:
X = range(-4, stop = 4, length = 128)
Y = range(-4, stop = 4, length = 128) 

t = 3.0
Z = [ E([x y], t ) for y in Y, x in X ];
heatmap(X, Y, log10.(Z))

In [ ]:
V₀ = v₀ₓ(0)
V² = V₀ ⋅ V₀
Vn = normalize(V₀)

heatmap(X, Y, clamp.(log10.(Z), -1, 4), # oбрезание
    legend = false, ticks = nothing, border = :none, size = (600, 400), c = :devon  ) # :grays

for alph in π/6:π/6:2π
    nn = [cos(alph) sin(alph)]
    fvec = (sqrt(1-V²)-1) * (Vn ⋅ nn) * Vn + nn
    fi = angle(fvec[1] + im*fvec[2])
    
    Vv = V₀ ⋅ [cos(fi) sin(fi)]
    μ = Vv + sqrt( 1-V² + Vv^2 )
    fvec = V₀ - μ*[cos(fi) sin(fi)]
    fi = angle(fvec[1] + im*fvec[2])
    
    zz = vcat( [λ(t, tt, fi) for tt in t-6:0.01:t ]... );
    plot!(zz[:,1], zz[:,2], line = (1, :yellow), )
end
scatter!([x₀(t)[1]],[x₀(t)[2]], c = :red)
yaxis!((Y[1], Y[end]))
xaxis!((X[1], X[end]))

In [ ]:
@time for (i, t) in enumerate(range(0, stop = 3, length = 16)) # картинки для гифки
    Z = [ E([x y], t) for y in Y, x in X ];
    heatmap(X, Y, clamp.(log10.(Z), -1, 4), # oбрезание
        legend = false, ticks = nothing, border = :none, size = (600, 400)  )

    for alph in π/4:π/4:2π
        nn = [cos(alph) sin(alph)]
        fvec = (sqrt(1-V²)-1) * (Vn ⋅ nn) * Vn + nn
        fi = angle(fvec[1] + im*fvec[2])

        Vv = V₀ ⋅ [cos(fi) sin(fi)]
        μ = Vv + sqrt( 1-V² + Vv^2 )
        fvec = V₀ - μ*[cos(fi) sin(fi)]
        fi = angle(fvec[1] + im*fvec[2])

        zz = vcat( [λ(t, tt, fi) for tt in t-6:0.005:t ]... );
        plot!(zz[:,1], zz[:,2], line = (1, :yellow), )
    end
    scatter!([x₀(t)[1]],[x₀(t)[2]], c = :red)
    yaxis!((Y[1], Y[end]))
    xaxis!((X[1], X[end]))

    savefig("img$i.png")
end

## Задания

1. Разобрать приведенные листинги и создать анимации для останавливающейся, осциллирующей и движущейся по окружности заряженной частицы.
1. Визуализироваль силовые линии и волны в ЭП для ускоряющегося заряда и ондулятора.
1. Визуализироваль силовые линии и волны в ЭП для произвольного нетривиального движения.
1. * Создать интерактивный визуализатор с управлением мышью (Произвольные средства разработки или пакет [Makie.jl](https://github.com/JuliaPlots/Makie.jl)).